# Multi-task recommenders

**Learning Objectives**
  1. Training a model which focuses on ratings.
  2. Training a model which focuses on retrieval.
  3. Training a joint model that assigns positive weights to both ratings & retrieval models.


## Introduction
In the basic retrieval notebook we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In addition, building a joint model for multiple tasks may produce better results than building a number of task-specific models. This is especially true where some data is abundant (for example, clicks), and some data is sparse (purchases, returns, manual reviews). In those scenarios, a joint model may be able to use representations learned from the abundant task to improve its predictions on the sparse task via a phenomenon known as [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). For example, [this paper](https://openreview.net/pdf?id=SJxPVcSonN) shows that a model predicting explicit user ratings from sparse user surveys can be substantially improved by adding an auxiliary task that uses abundant click log data.

In this jupyter notebook, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/recommendation_systems/labs/multitask.ipynb) -- try to complete that notebook first before reviewing this solution notebook.


## Imports


Let's first get our imports out of the way.


In [2]:
# Installing the necessary libraries.
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

**NOTE: Please ignore any incompatibility warnings and errors and re-run the above cell before proceeding.**


In [3]:
# Importing the necessary modules
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

We're going to use the Movielens 100K dataset.

In [5]:
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
movies = movies.map(lambda x: x["movie_title"])

And repeat our preparations for building vocabularies and splitting the data into a train and a test set:

In [6]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## A multi-task model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In this jupyter notebook, we will define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

```python
user_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add 1 to account for the unknown token.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])
```

However, now we will have two tasks. The first is the rating task:

```python
tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)
```

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

```python
tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128)
    )
)
```

As before, this task's goal is to predict which movies the user will or will not watch.

### Putting it together

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [7]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["movie_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("user_rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [8]:
# Here, configuring the model with losses and metrics.
# TODO 1: Here is your code.
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [9]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [10]:
# Training the ratings model.
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3


 1/10 [==>...........................] - ETA: 0s - root_mean_squared_error: 3.6922 - factorized_top_k/top_1_categorical_accuracy: 4.8828e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0031 - factorized_top_k/top_10_categorical_accuracy: 0.0060 - factorized_top_k/top_50_categorical_accuracy: 0.0294 - factorized_top_k/top_100_categorical_accuracy: 0.0575 - loss: 13.6320 - regularization_loss: 0.0000e+00 - total_loss: 13.6320

 2/10 [=====>........................] - ETA: 1s - root_mean_squared_error: 3.2709 - factorized_top_k/top_1_categorical_accuracy: 5.4932e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0031 - factorized_top_k/top_10_categorical_accuracy: 0.0058 - factorized_top_k/top_50_categorical_accuracy: 0.0298 - factorized_top_k/top_100_categorical_accuracy: 0.0587 - loss: 10.6989 - regularization_loss: 0.0000e+00 - total_loss: 10.6989

 3/10 [========>.....................] - ETA: 1s - root_mean_squared_error: 2.8313 - factorized_top_k/top_1_categorical_accuracy: 5.2897e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0030 - factorized_top_k/top_10_categorical_accuracy: 0.0059 - factorized_top_k/top_50_categorical_accuracy: 0.0308 - factorized_top_k/top_100_categorical_accuracy: 0.0594 - loss: 8.0165 - regularization_loss: 0.0000e+00 - total_loss: 8.0165  

 4/10 [===========>..................] - ETA: 1s - root_mean_squared_error: 2.8111 - factorized_top_k/top_1_categorical_accuracy: 4.5776e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0027 - factorized_top_k/top_10_categorical_accuracy: 0.0059 - factorized_top_k/top_50_categorical_accuracy: 0.0302 - factorized_top_k/top_100_categorical_accuracy: 0.0597 - loss: 7.9023 - regularization_loss: 0.0000e+00 - total_loss: 7.9023

...

3/5 [=================>............] - ETA: 0s - root_mean_squared_error: 1.1200 - factorized_top_k/top_1_categorical_accuracy: 6.5104e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0028 - factorized_top_k/top_10_categorical_accuracy: 0.0046 - factorized_top_k/top_50_categorical_accuracy: 0.0280 - factorized_top_k/top_100_categorical_accuracy: 0.0578 - loss: 1.2543 - regularization_loss: 0.0000e+00 - total_loss: 1.2543

4/5 [=======================>......] - ETA: 0s - root_mean_squared_error: 1.1156 - factorized_top_k/top_1_categorical_accuracy: 5.4932e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0030 - factorized_top_k/top_10_categorical_accuracy: 0.0050 - factorized_top_k/top_50_categorical_accuracy: 0.0294 - factorized_top_k/top_100_categorical_accuracy: 0.0596 - loss: 1.2447 - regularization_loss: 0.0000e+00 - total_loss: 1.2447

5/5 [==============================] - ETA: 0s - root_mean_squared_error: 1.1130 - factorized_top_k/top_1_categorical_accuracy: 5.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0030 - factorized_top_k/top_10_categorical_accuracy: 0.0053 - factorized_top_k/top_50_categorical_accuracy: 0.0298 - factorized_top_k/top_100_categorical_accuracy: 0.0595 - loss: 1.2380 - regularization_loss: 0.0000e+00 - total_loss: 1.2380

5/5 [==============================] - 1s 107ms/step - root_mean_squared_error: 1.1130 - factorized_top_k/top_1_categorical_accuracy: 5.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0030 - factorized_top_k/top_10_categorical_accuracy: 0.0053 - factorized_top_k/top_50_categorical_accuracy: 0.0298 - factorized_top_k/top_100_categorical_accuracy: 0.0595 - loss: 1.2336 - regularization_loss: 0.0000e+00 - total_loss: 1.2336


Retrieval top-100 accuracy: 0.060.
Ranking RMSE: 1.113.


The model does OK on predicting ratings (with an RMSE of around 1.11), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 4 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [11]:
# Here, configuring the model with losses and metrics.
# TODO 2: Here is your code.
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [12]:
# Training the retrieval model.
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3


 1/10 [==>...........................] - ETA: 0s - root_mean_squared_error: 3.6975 - factorized_top_k/top_1_categorical_accuracy: 3.6621e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0032 - factorized_top_k/top_10_categorical_accuracy: 0.0061 - factorized_top_k/top_50_categorical_accuracy: 0.0288 - factorized_top_k/top_100_categorical_accuracy: 0.0581 - loss: 73817.5703 - regularization_loss: 0.0000e+00 - total_loss: 73817.5703

 2/10 [=====>........................] - ETA: 1s - root_mean_squared_error: 3.7038 - factorized_top_k/top_1_categorical_accuracy: 5.4932e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0046 - factorized_top_k/top_10_categorical_accuracy: 0.0087 - factorized_top_k/top_50_categorical_accuracy: 0.0372 - factorized_top_k/top_100_categorical_accuracy: 0.0731 - loss: 73818.4492 - regularization_loss: 0.0000e+00 - total_loss: 73818.4492

 3/10 [========>.....................] - ETA: 1s - root_mean_squared_error: 3.7059 - factorized_top_k/top_1_categorical_accuracy: 6.9173e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0050 - factorized_top_k/top_10_categorical_accuracy: 0.0100 - factorized_top_k/top_50_categorical_accuracy: 0.0444 - factorized_top_k/top_100_categorical_accuracy: 0.0821 - loss: 73819.6068 - regularization_loss: 0.0000e+00 - total_loss: 73819.6068

 4/10 [===========>..................] - ETA: 1s - root_mean_squared_error: 3.7098 - factorized_top_k/top_1_categorical_accuracy: 8.5449e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0057 - factorized_top_k/top_10_categorical_accuracy: 0.0114 - factorized_top_k/top_50_categorical_accuracy: 0.0515 - factorized_top_k/top_100_categorical_accuracy: 0.0932 - loss: 73815.9902 - regularization_loss: 0.0000e+00 - total_loss: 73815.9902

...

3/5 [=================>............] - ETA: 0s - root_mean_squared_error: 3.7744 - factorized_top_k/top_1_categorical_accuracy: 7.3242e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0090 - factorized_top_k/top_10_categorical_accuracy: 0.0215 - factorized_top_k/top_50_categorical_accuracy: 0.1253 - factorized_top_k/top_100_categorical_accuracy: 0.2347 - loss: 32516.4388 - regularization_loss: 0.0000e+00 - total_loss: 32516.4388

4/5 [=======================>......] - ETA: 0s - root_mean_squared_error: 3.7732 - factorized_top_k/top_1_categorical_accuracy: 0.0010 - factorized_top_k/top_5_categorical_accuracy: 0.0093 - factorized_top_k/top_10_categorical_accuracy: 0.0217 - factorized_top_k/top_50_categorical_accuracy: 0.1246 - factorized_top_k/top_100_categorical_accuracy: 0.2359 - loss: 32495.7471 - regularization_loss: 0.0000e+00 - total_loss: 32495.7471    

5/5 [==============================] - ETA: 0s - root_mean_squared_error: 3.7730 - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0098 - factorized_top_k/top_10_categorical_accuracy: 0.0218 - factorized_top_k/top_50_categorical_accuracy: 0.1253 - factorized_top_k/top_100_categorical_accuracy: 0.2353 - loss: 31649.3406 - regularization_loss: 0.0000e+00 - total_loss: 31649.3406

5/5 [==============================] - 1s 108ms/step - root_mean_squared_error: 3.7730 - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0098 - factorized_top_k/top_10_categorical_accuracy: 0.0218 - factorized_top_k/top_50_categorical_accuracy: 0.1253 - factorized_top_k/top_100_categorical_accuracy: 0.2353 - loss: 31085.0697 - regularization_loss: 0.0000e+00 - total_loss: 31085.0697


Retrieval top-100 accuracy: 0.235.
Ranking RMSE: 3.773.


We get the opposite result: a model that does well on retrieval, but poorly on predicting ratings.

### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [13]:
# Here, configuring the model with losses and metrics.
# TODO 3: Here is your code.
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [14]:
# Training the joint model.
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3


 1/10 [==>...........................] - ETA: 0s - root_mean_squared_error: 3.6615 - factorized_top_k/top_1_categorical_accuracy: 2.4414e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0028 - factorized_top_k/top_10_categorical_accuracy: 0.0060 - factorized_top_k/top_50_categorical_accuracy: 0.0320 - factorized_top_k/top_100_categorical_accuracy: 0.0636 - loss: 73830.4453 - regularization_loss: 0.0000e+00 - total_loss: 73830.4453

 2/10 [=====>........................] - ETA: 0s - root_mean_squared_error: 3.1084 - factorized_top_k/top_1_categorical_accuracy: 7.9346e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0092 - factorized_top_k/top_50_categorical_accuracy: 0.0403 - factorized_top_k/top_100_categorical_accuracy: 0.0735 - loss: 73829.0508 - regularization_loss: 0.0000e+00 - total_loss: 73829.0508

 3/10 [========>.....................] - ETA: 1s - root_mean_squared_error: 3.0812 - factorized_top_k/top_1_categorical_accuracy: 8.1380e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0052 - factorized_top_k/top_10_categorical_accuracy: 0.0101 - factorized_top_k/top_50_categorical_accuracy: 0.0459 - factorized_top_k/top_100_categorical_accuracy: 0.0830 - loss: 73829.5286 - regularization_loss: 0.0000e+00 - total_loss: 73829.5286

 4/10 [===========>..................] - ETA: 1s - root_mean_squared_error: 3.1885 - factorized_top_k/top_1_categorical_accuracy: 8.5449e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0058 - factorized_top_k/top_10_categorical_accuracy: 0.0114 - factorized_top_k/top_50_categorical_accuracy: 0.0537 - factorized_top_k/top_100_categorical_accuracy: 0.0958 - loss: 73825.9199 - regularization_loss: 0.0000e+00 - total_loss: 73825.9199

...

3/5 [=================>............] - ETA: 0s - root_mean_squared_error: 1.1364 - factorized_top_k/top_1_categorical_accuracy: 8.1380e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0087 - factorized_top_k/top_10_categorical_accuracy: 0.0222 - factorized_top_k/top_50_categorical_accuracy: 0.1236 - factorized_top_k/top_100_categorical_accuracy: 0.2321 - loss: 32499.3945 - regularization_loss: 0.0000e+00 - total_loss: 32499.3945

4/5 [=======================>......] - ETA: 0s - root_mean_squared_error: 1.1337 - factorized_top_k/top_1_categorical_accuracy: 9.1553e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0085 - factorized_top_k/top_10_categorical_accuracy: 0.0214 - factorized_top_k/top_50_categorical_accuracy: 0.1237 - factorized_top_k/top_100_categorical_accuracy: 0.2335 - loss: 32477.6860 - regularization_loss: 0.0000e+00 - total_loss: 32477.6860

5/5 [==============================] - ETA: 0s - root_mean_squared_error: 1.1312 - factorized_top_k/top_1_categorical_accuracy: 8.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0087 - factorized_top_k/top_10_categorical_accuracy: 0.0219 - factorized_top_k/top_50_categorical_accuracy: 0.1248 - factorized_top_k/top_100_categorical_accuracy: 0.2344 - loss: 31628.7672 - regularization_loss: 0.0000e+00 - total_loss: 31628.7672

5/5 [==============================] - 1s 111ms/step - root_mean_squared_error: 1.1312 - factorized_top_k/top_1_categorical_accuracy: 8.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0087 - factorized_top_k/top_10_categorical_accuracy: 0.0219 - factorized_top_k/top_50_categorical_accuracy: 0.1248 - factorized_top_k/top_100_categorical_accuracy: 0.2344 - loss: 31062.8213 - regularization_loss: 0.0000e+00 - total_loss: 31062.8213


Retrieval top-100 accuracy: 0.234.
Ranking RMSE: 1.131.


The result is a model that performs roughly as well on both tasks as each specialized model. 

While the results here do not show a clear accuracy benefit from a joint model in this case, multi-task learning is in general an extremely useful tool. We can expect better results when we can transfer knowledge from a data-abundant task (such as clicks) to a closely related data-sparse task (such as purchases).